In [40]:
import pandas as pd
import plotly
#import chart_studio.plotly as py
import math
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.core.display import display, HTML
%matplotlib inline
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)

In [89]:
xls = pd.ExcelFile('Matching Lanes.xlsx')
hist_rates = pd.read_excel(xls, 'Historical Rates')
truck_wiz = pd.read_excel(xls, 'TruckWiz Vans')

In [90]:
hist_rates.rename(columns={'origin_zip_code':'OZip','dest_zip_code':'DZip'}, inplace=True)

In [91]:
hist_rates.head()

,Customer Name,Account Name,IsCurrent,Origin,OZip,Destination,DZip
0,1 800 PACK RAT,1 800 PACK RAT,(blank),"BREWTON, AL",36426,"DUBUQUE, IA",52001
1,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"NABB, IN",47147,"HUNTINGDON, TN",38344
2,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"HUNTINGDON, TN",38344
3,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"LORAIN, OH",44055
4,ABSOLUTE FREIGHT MANAGEMENT,ABSOLUTE FREIGHT MANAGEMENTE,(blank),"DEPEW, NY",14043,"LAWRENCEBURG, TN",38464


In [92]:
truck_wiz=truck_wiz[['SHIPPER..................','OS', 'OCTY......', 'DS', 'DCTY......','MVES', 'RPBM-',
       'FSUR', 'LMLS', 'LDYS', 'EMLS', 'EDYS', 'OVCS', 'MTYV-', 'TMEV-',
       'LDSV-', 'FRTQ-', 'ZIP1..', 'ZIP2..', 'ZIP3..']]

In [93]:
truck_wiz.rename(columns={'SHIPPER..................':'Shipper', 'OCTY......':'OCity', 'DCTY......':'DCity','RPBM-':'RPBM',
                       'MTYV-':'MTYV','TMEV-':'TMEV','LDSV-':'LDSV','FRTQ-':'FRQT','ZIP2..':'OZip','ZIP3..':'DZip'}, inplace=True)


In [94]:
truck_wiz.head()

,Shipper,OS,OCity,DS,DCity,MVES,RPBM,FSUR,LMLS,LDYS,EMLS,EDYS,OVCS,MTYV,TMEV,LDSV,FRQT,ZIP1..,OZip,DZip
0,b: C H ROBINSON |,AL,EASTABOGA,IN,HUNTINGBUR,8,1.46,0.27,410,1.07,95,0.33,-24,-319,-44,102,-261,35649,36260,47542
1,b: C H ROBINSON |,AL,EASTABOGA,KY,MOUNT STER,1,1.41,0.27,417,0.55,103,0.35,-25,-319,80,-53,-292,36830,36260,40353
2,b: C H ROBINSON |,AL,EASTABOGA,KY,SOMERSET,33,1.69,0.27,306,1.09,93,0.31,-19,-319,-61,-24,-404,35055,36260,42501
3,LOWES MLXP VAN ONE WA|,AL,CALERA,GA,NEWNAN,1,3.18,0.21,150,0.20,80,0.30,0,0,51,337,388,35055,35040,30263
4,b: C H ROBINSON |,AL,TRUSSVILLE,IL,GURNEE,1,1.17,0.27,727,1.96,116,0.35,-2,-140,-83,-66,-289,36830,35173,60031


In [7]:
# truck_wiz['Shipper']=truck_wiz['Shipper'].str.replace('b:', '', regex=True)
# truck_wiz['Shipper']=truck_wiz['Shipper'].str.replace('|', '', regex=True)
# truck_wiz['Shipper']=truck_wiz['Shipper'].str.strip()
# truck_wiz['Shipper']=truck_wiz['Shipper'].str.replace(" ","")

In [10]:
# hist_rates['Account Name']=hist_rates['Account Name'].str.replace('b:', '', regex=True)
# hist_rates['Account Name']=hist_rates['Account Name'].str.replace('|', '', regex=True)
# hist_rates['Account Name']=hist_rates['Account Name'].str.strip()
# hist_rates['Account Name']=hist_rates['Account Name'].str.replace(" ","")
# #hist_rates['Account Name1']=hist_rates['Account Name'].map(lambda x: str(x)[:-1])


In [106]:
hist_rates['OZip'] = hist_rates['OZip'].astype(str)
hist_rates['DZip'] = hist_rates['DZip'].astype(str)

In [107]:
truck_wiz['OZip'] = truck_wiz['OZip'].astype(str)
truck_wiz['DZip'] = truck_wiz['DZip'].astype(str)

In [16]:
# truck_wiz['Lane']=truck_wiz['Shipper'].str.cat([truck_wiz['OZip'],truck_wiz['DZip']], sep=',')
# truck_wiz['Lane'].head()

0           CHROBINSON,36260,47542
1           CHROBINSON,36260,40353
2           CHROBINSON,36260,42501
3    LOWESMLXPVANONEWA,35040,30263
4           CHROBINSON,35173,60031
Name: Lane, dtype: object

In [51]:
import pandas as pd 
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [53]:
names_array,ratio_array=checker(hist_rates['Account Name'],truck_wiz['Shipper'])

In [54]:
dataset2 = pd.DataFrame({'truck_wiz_name': names_array, 'ratio': ratio_array,'hist_rate_name':hist_rates['Account Name']})

In [98]:
dataset2

,truck_wiz_name,ratio,hist_rate_name
0,b: PATHMARK TRANSPORT|,47,1 800 PACK RAT
1,1876 LOGISTICS LLC |,100,1876 LOGISTICS LLC
2,1876 LOGISTICS LLC |,100,1876 LOGISTICS LLC
3,1876 LOGISTICS LLC |,100,1876 LOGISTICS LLC
4,BJN FREIGHT |,78,ABSOLUTE FREIGHT MANAGEMENTE
...,...,...,...
5646,b: ZENGISTICS INC |,100,ZENGISTICS INC
5647,b: ZENGISTICS INC |,100,ZENGISTICS INC
5648,b: ZIPLINE LOGISTICS |,100,ZIPLINE LOGISTICS
5649,KODIAK TRANSPORTATION|,85,ZUUM TRANSPORTATION


In [99]:
hist_rates['Shipper']=dataset2['truck_wiz_name']
hist_rates['score']=dataset2['ratio']

In [100]:
hist_rates.head()

,Customer Name,Account Name,IsCurrent,Origin,OZip,Destination,DZip,Shipper,score
0,1 800 PACK RAT,1 800 PACK RAT,(blank),"BREWTON, AL",36426,"DUBUQUE, IA",52001,b: PATHMARK TRANSPORT|,47
1,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"NABB, IN",47147,"HUNTINGDON, TN",38344,1876 LOGISTICS LLC |,100
2,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"HUNTINGDON, TN",38344,1876 LOGISTICS LLC |,100
3,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"LORAIN, OH",44055,1876 LOGISTICS LLC |,100
4,ABSOLUTE FREIGHT MANAGEMENT,ABSOLUTE FREIGHT MANAGEMENTE,(blank),"DEPEW, NY",14043,"LAWRENCEBURG, TN",38464,BJN FREIGHT |,78


In [101]:
truck_wiz.head()

,Shipper,OS,OCity,DS,DCity,MVES,RPBM,FSUR,LMLS,LDYS,EMLS,EDYS,OVCS,MTYV,TMEV,LDSV,FRQT,ZIP1..,OZip,DZip
0,b: C H ROBINSON |,AL,EASTABOGA,IN,HUNTINGBUR,8,1.46,0.27,410,1.07,95,0.33,-24,-319,-44,102,-261,35649,36260,47542
1,b: C H ROBINSON |,AL,EASTABOGA,KY,MOUNT STER,1,1.41,0.27,417,0.55,103,0.35,-25,-319,80,-53,-292,36830,36260,40353
2,b: C H ROBINSON |,AL,EASTABOGA,KY,SOMERSET,33,1.69,0.27,306,1.09,93,0.31,-19,-319,-61,-24,-404,35055,36260,42501
3,LOWES MLXP VAN ONE WA|,AL,CALERA,GA,NEWNAN,1,3.18,0.21,150,0.20,80,0.30,0,0,51,337,388,35055,35040,30263
4,b: C H ROBINSON |,AL,TRUSSVILLE,IL,GURNEE,1,1.17,0.27,727,1.96,116,0.35,-2,-140,-83,-66,-289,36830,35173,60031


In [108]:
result1=pd.merge(hist_rates,truck_wiz,on=['Shipper','OZip','DZip'],how='left')

In [109]:
result1

,Customer Name,Account Name,IsCurrent,Origin,OZip,Destination,DZip,Shipper,score,OS,...,LMLS,LDYS,EMLS,EDYS,OVCS,MTYV,TMEV,LDSV,FRQT,ZIP1..
0,1 800 PACK RAT,1 800 PACK RAT,(blank),"BREWTON, AL",36426,"DUBUQUE, IA",52001,b: PATHMARK TRANSPORT|,47,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"NABB, IN",47147,"HUNTINGDON, TN",38344,1876 LOGISTICS LLC |,100,IN,...,305.0,0.71,33.0,0.25,-5.0,27.0,29.0,245.0,301.0,47265.0
2,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"HUNTINGDON, TN",38344,1876 LOGISTICS LLC |,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1876 LOGISTICS,1876 LOGISTICS LLC,(blank),"WILLS POINT, TX",75169,"LORAIN, OH",44055,1876 LOGISTICS LLC |,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABSOLUTE FREIGHT MANAGEMENT,ABSOLUTE FREIGHT MANAGEMENTE,(blank),"DEPEW, NY",14043,"LAWRENCEBURG, TN",38464,BJN FREIGHT |,78,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5661,Zengistics,ZENGISTICS INC,(blank),"ATHENS, GA",30601,"NICHOLASVILLE, KY",40356,b: ZENGISTICS INC |,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5662,Zengistics,ZENGISTICS INC,(blank),"PLAINVILLE, GA",30733,"OLIVE BRANCH, MS",38654,b: ZENGISTICS INC |,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5663,Zipline Logistics,ZIPLINE LOGISTICS,(blank),"DELAWARE, OH",43015,"MAYFIELD, KY",42066,b: ZIPLINE LOGISTICS |,100,OH,...,460.0,0.96,39.0,0.25,-28.0,13.0,1.0,-88.0,-74.0,43125.0
5664,ZUUM TRANSPORTATION,ZUUM TRANSPORTATION,(blank),"MIDDLETOWN, NY",10941,"RUSSELLVILLE, KY",42276,KODIAK TRANSPORTATION|,85,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
with pd.ExcelWriter('matchoutput.xlsx') as writer:  
     result1.to_excel(writer, sheet_name='Mapped on Python')

In [110]:
result1['FRQT'].isnull().sum()

4821